![](https://www.math.unipd.it/~marcuzzi/BannerStrumentifondamentali.png)

# Moltiplicazione tra matrici

In questo kernel vedremo come utilizzare la shared memory in un problema di algebra lineare. Date due matrici $A$ e $B$ di dimensione rispettivamente $m \times k$ e $k \times n$, vogliamo scrivere un kernel per calcolare il prodotto $C = AB$.

In [1]:
import numpy as np
import numba
from numba import cuda
import math
from timeit import default_timer
from matplotlib import pyplot as plt
#from matplotlib import colors
%matplotlib inline

In [2]:
m = 128
k = 256
n = 128

A = np.random.randint(10, size = (m,k), dtype = 'int32')
B = np.random.randint(10, size = (k,n), dtype = 'int32')

In [3]:
t_i  = default_timer()
C_np = A@B
t_f  = default_timer()
print("Tempo di esecuzione = ", t_f - t_i, "s")

Tempo di esecuzione =  0.0034603364765644073 s


In [4]:
def matmul_py(A,B):
    if (B.shape[0] != A.shape[1]):
        print("Errore: le dimensioni delle matrici non sono compatibili!")
        return
    
    C = np.zeros((m,n))
    
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            tmp = 0
            for k in range(A.shape[1]):
                tmp = tmp + A[i,k]*B[k,j]
            #endfor
            C[i,j] = tmp
        #endfor
    #endfor
    
    return C

In [5]:
t_i  = default_timer()
C_py = matmul_py(A,B)
t_f  = default_timer()
print('|| C_np - C_py ||', np.linalg.norm(C_np-C_py))
print("Tempo di esecuzione = ", t_f - t_i, "s")

|| C_np - C_py || 0.0
Tempo di esecuzione =  1.8731305487453938 s


Nella cella seguente troviamo un'implementazione naif di un prodotto tra matrici su GPU. Assegnamo al thread $(i,j)$ il calcolo dell'elemento 

$$C_{ij} = \sum_k A_{i,k}B_{kj} = A_{i:} \cdot B_{:j},$$

che altro non è che il prodotto interno dell'$i$-esima riga di $A$, $A_{i:}$, e della $j$-esima colonna di $B$, $B_{:j}$. Senza tener conto della memoria shared, ogni thread legge una righa di $A$ e una colonna di $B$, come illustrato nella figura sotto. 

![](https://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/matrix-multiplication-without-shared-memory.png)

Notiamo che in questo modo la matrice $A$ viene letta $n$ (`B.width`) volte dalla global memory e la matrice $B$ viene letta $m$ (`A.height`) volte. 

In [6]:
@cuda.jit()
def matmul_gpu(A,B,C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        #endfor
        C[i, j] = tmp
    #endif
    return 

In [7]:
TPB = 32
threadsperblock = (TPB, TPB)
blockspergrid_x = math.ceil(A.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(B.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

C_gpu = np.zeros((m,n), dtype = np.int32)

matmul_gpu[blockspergrid, threadsperblock](A,B,C_gpu)
print('|| C_np - C_gpu ||', np.linalg.norm(C_np-C_gpu))

|| C_np - C_gpu || 0.0


In [8]:
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C_gpu = cuda.device_array_like(C_gpu)

t_i = default_timer()
matmul_gpu[blockspergrid, threadsperblock](d_A,d_B,d_C_gpu)
cuda.synchronize()
t_f = default_timer()
print("Tempo di esecuzione = ", t_f - t_i, "s" )

del  d_A, d_B, d_C_gpu

Tempo di esecuzione =  0.21100536733865738 s


## Shared memory e tiling

Vediamo l'implementazione con la memoria shared. Suddividiamo la matrice $C$ in sottomatrici quadrate $C_{sub}$.  Come illustrato nella figura sotto, la sottomatrice $C_{sub}$ è il prodotto di due sottomatrici rettangolari. Assegnamo ogni blocco di thread al calcolo di una sottomatrice quadrata $C_{sub}$ di $C$ e ogni thread all'interno del blocco al calcolo di un elemento di $C_{sub}$.
Notiamo che così facendo, ogni blocco è indipendente dagli altri. Questo requisito è fondamentale se vogliamo implementare un algoritmo sulla GPU. Notiamo anche come a livello di blocchi si ha un palallelismo a grana grossa e come all'interno di ogni blocco si abbia un parallelismo a grana fine dato dai thread  paralleli. Nella memoria shared dovremo cacaricare la sottomatrice di $A$ di dimensione `(A.width, block_size)` e la sottomatrice di $B$ di dimensione `(block_size, A.width)`.
La memoria shared è molto più piccola della memoria global, e le matrici potrebbero essere troppo grandi per essere caricate in shared, perciò è necessario suddividerle in un numero oppurtuno di sottomatrici quadrate di dimensione `(block_size, block_size)` dove `block_size`è la dimensione dei blocchi. $C_{sub}$ viene calcolata come una somma incrementale di prodotti di sottomatrici. Ognuno di questi prodotti viene effettuato caricando prima le corrispondenti sottomatrici quadrate di $A$ e $B$ in memoria shared (ogni thread si occupa di caricare un elemento della sottomatrice di $A$ e uno della sottomatrice di $B$), dopodiché ogni thread si occupa del calcolo di un elemento del prodotto. Ogni thread accumula i risultati in una somma parziale (una varialibe in memoria locale), e una volta terminate le piastrelle scrive il risultato nella memoria global. Questa tecnica prende il come di **tiling**, dall'inglese *tile*, "piastrella".

![](https://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/matrix-multiplication-with-shared-memory.png)

Questo approccio ci permette di minimizzare il numero delle letture di $A$ e $B$ dalla memoria global: $A$ viene letta solo `(B.width / block_size)` volte e $B$ viene letta `(A.height / block_size)`. 

In [9]:
TPB = 32
@cuda.jit()
def matmul_gpu_shared(A, B, C):
    # Controllo che le matrici A e B siano compatibili per il prodotto
    n = A.shape[1]
    if (B.shape[0] != n):
        return
    
    # Definisco array shared per caricare le piastrelle
    # La dimensione e il tipo devono essere noti al momento della compilazione
    A_sh = cuda.shared.array(shape=(TPB, TPB), dtype=numba.int32)
    B_sh = cuda.shared.array(shape=(TPB, TPB), dtype=numba.int32)
    i, j = cuda.grid(2) #indici per accedere alla memoria globale

    s_i = cuda.threadIdx.x #indice locale x per accedere alla memoria shared
    s_j = cuda.threadIdx.y #indice locale y per accedere alla memoria shared
    Ntiles = math.ceil(n/TPB) # numero di piastrelle lungo la direzione comune di A e B

    # Ogni thread calcola un elemento della matrice C.
    # Il prodotto interno tra le righe e le colonne viene spezzato nel prodotto interno di vettori di lunghezza TPB
    if (i < C.shape[0]) and (j < C.shape[1]):
        tmp = 0
        for k in range(Ntiles):
            # Caricamento dati in shared memory
            if (s_j + k * TPB < n) and (s_i + k * TPB < n):
                A_sh[s_i, s_j] = A[i, s_j + k * TPB]
                B_sh[s_i, s_j] = B[s_i + k * TPB, j]
            else:
                A_sh[s_i, s_j] = 0
                B_sh[s_i, s_j] = 0
            #endif
        
            # Barriera: attendo che tutti i threads abbiano terminato il caricamento in shared
            cuda.syncthreads()

            # Calcolo il prodotto parziale nella shared memory
            for l in range(TPB):
                tmp += A_sh[s_i, l] * B_sh[l, s_j]

            # Barriera: attendo che tutti i threads abbiano terminato il calcolo
            cuda.syncthreads()
        #endif
    
        #scrivo il risultato in global memory
        C[i, j] = tmp
    #endif
    return

In [10]:
threadsperblock = (TPB, TPB)
blockspergrid_x = math.ceil(A.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(B.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

matmul_gpu_shared[blockspergrid, threadsperblock](A,B,C_gpu)
print('|| C_np - C_gpu ||', np.linalg.norm(C_np-C_gpu))

|| C_np - C_gpu || 0.0


In [11]:
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C_gpu = cuda.device_array_like(C_np)

t_i = default_timer()
matmul_gpu_shared[blockspergrid, threadsperblock](d_A,d_B,d_C_gpu)
cuda.synchronize()
t_f = default_timer()
print("Tempo di esecuzione = ", t_f - t_i, "s")

del  d_A, d_B, d_C_gpu

Tempo di esecuzione =  0.2962923627346754 s


In [12]:
from numba import guvectorize, void, int32

@guvectorize(['int32[:,:], int32[:,:], int32[:,:]'], 
             '(m,k),(k,n)->(m,n)', target='cuda')
def matmul_gu(A, B, C):
    m, k = A.shape
    k, n = B.shape
    for i in range(m):
        for j in range(n):
            tmp = 0
            for l in range(k):
                tmp += A[i, l] * B[l, j]
            #endfor
            C[i, j] = tmp
        #endfor
    #endfor
    return
 
matmul_gu.max_blocksize = (32,32)

In [13]:
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C_gu = cuda.device_array_like(C_np)

t_i = default_timer()
matmul_gu(d_A, d_B, d_C_gu)
cuda.synchronize()
t_f = default_timer()
C_gu = d_C_gu.copy_to_host()
print('|| C_np - C_gu ||', np.linalg.norm(C_np - C_gu))
print("Tempo di esecuzione = ", t_f - t_i, "s")

del d_A, d_B, d_C_gu

|| C_np - C_gu || 0.0
Tempo di esecuzione =  0.30342525243759155 s


In [14]:
#TEST
k = 4
n = TPB * k
BPG = math.ceil(n/TPB)
block_dim = (TPB, TPB)
grid_dim = (BPG, BPG)

# Prepare data on the CPU
A = np.array(np.random.random((n, n)), dtype=np.int32, order = 'F')
B = np.array(np.random.random((n, n)), dtype=np.int32, order = 'F')
C = np.zeros_like(A)

print("Dimensione matrici = %d x %d" % (n, n))

# Prepare data on the GPU
dA = cuda.to_device(A)
dB = cuda.to_device(B)
dC = cuda.to_device(C) # device_array_like(A)

# Time numpy version
t_i = default_timer()
np_ans = np.dot(A, B)
t_f = default_timer()
t_np = t_f - t_i 

# Time the unoptimized version
t_i = default_timer()
matmul_gpu[grid_dim, block_dim](dA, dB, dC)
cuda.synchronize()
t_f = default_timer()
tcuda = t_f - t_i 


# Time the shared memory version
t_i = default_timer()
matmul_gpu_shared[grid_dim, block_dim](dA, dB, dC)
cuda.synchronize()
t_f = default_timer()
tcuda_sh = t_f - t_i 

# Time guvectorize version
t_i = default_timer()
matmul_gu(dA, dB, dC)
cuda.synchronize()
t_f = default_timer()
t_gu = t_f - t_i 

print("Numpy:", "%.17f" % t_np, "s")
print("CUDA senza shared memory:", "%.17f" % tcuda, "s")
print("CUDA con shared memory:", "%.17f" % tcuda_sh, "s")
print("guvectoruze:", "%.17f" % t_gu, "s")

del dA, dB, dC

Dimensione matrici = 128 x 128
Numpy: 0.00191385485231876 s
CUDA senza shared memory: 0.00044769421219826 s
CUDA con shared memory: 0.00047342479228973 s
guvectoruze: 0.13595477305352688 s
